In [37]:
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
import azapy as az

In [38]:
sdate = pd.to_datetime("2012-01-01")
edate = pd.to_datetime("today")
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']

mktdir = "./MkTdata"

# force=True read from alphavantage server
# force=False try to read first from the local directory if data exists if not from alphavantage
mktdata = az.readMkT(symb, dstart = sdate, dend = edate, 
                     dir=mktdir, force=False) 

Read GLD form ./MkTdata
Read TLT form ./MkTdata
Read XLV form ./MkTdata
Read VGT form ./MkTdata
Read PSJ form ./MkTdata


In [39]:
alpha_CVaR98 = [0.96]
alpha_CVaR97 = [0.95]
alpha_CVaR95 = [0.94]
alpha_CVaR = alpha_CVaR98 + alpha_CVaR97 + alpha_CVaR95

coef_MAD1 = [1.]
coef_MAD2 = [0.5] * 2
coef_MAD3 = [1./3] * 3

rtype = 'InvNrisk'

hlength1 = 3.25

models = {'P_CVaR98': {'type': 'Port_CVaR', 'm_param': {'mu': 0., 'alpha': alpha_CVaR98, 'rtype': rtype, 'hlength': hlength1}},
          'P_CVaR97': {'type': 'Port_CVaR', 'm_param': {'mu': 0., 'alpha': alpha_CVaR97, 'rtype': rtype, 'hlength': hlength1}},
          'P_CVaR95': {'type': 'Port_CVaR', 'm_param': {'mu': 0., 'alpha': alpha_CVaR95, 'rtype': rtype, 'hlength': hlength1}},
          'P_CVaR': {'type': 'Port_CVaR', 'm_param': {'mu': 0., 'alpha': alpha_CVaR, 'rtype': rtype, 'hlength': hlength1}},
          'P_MAD1': {'type': 'Port_MAD', 'm_param': {'mu': 0., 'coef': coef_MAD1, 'rtype': rtype, 'hlength': hlength1}},
          'P_MAD2': {'type': 'Port_MAD', 'm_param': {'mu': 0., 'coef': coef_MAD2, 'rtype': rtype, 'hlength': hlength1}},
          'P_MAD3': {'type': 'Port_MAD', 'm_param': {'mu': 0., 'coef': coef_MAD3, 'rtype': rtype, 'hlength': hlength1}},
          'P_N': {'type': 'Port_ConstW', 'm_param': {'ww': None}}}


In [40]:
port = []
pp = {}
for key, val in models.items():
    ppz = getattr(az, val['type'])
    pp_ = ppz(mktdata, pname=key)
    pp[key] = pp_
    port_ = pp_.set_model(**val['m_param'])
    port.append(port_)

In [41]:
pdata = pd.concat(port, axis=1, join='inner').melt(var_name='symbol', value_name='close', ignore_index=False)

In [42]:
ps = az.Port_Simple(pdata, col='close', pname='ALL')
ports = ps.set_model()

In [43]:
_ = ps.port_view_all(sdate=pd.to_datetime('2000-01-01'), componly=True, fancy=True)

In [44]:
ps.port_perf(componly=True, fancy=True)

,RR,DD,DD_date,DD_start,DD_end
symbol,,,,,
P_CVaR,13.99,-19.52,2020-03-20,2020-02-19,2020-04-29
P_CVaR95,14.20,-19.71,2020-03-20,2020-02-19,2020-04-29
P_CVaR97,14.06,-19.51,2020-03-20,2020-02-19,2020-04-27
P_CVaR98,13.90,-19.56,2020-03-20,2020-02-19,2020-04-24
P_MAD1,13.56,-18.41,2020-03-20,2020-02-19,2020-04-24
P_MAD2,13.58,-18.83,2020-03-20,2020-02-19,2020-04-24
P_MAD3,13.84,-19.41,2020-03-20,2020-02-19,2020-04-29
P_N,15.72,-17.43,2020-03-18,2020-02-19,2020-04-14


In [45]:
ps.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,P_CVaR,P_CVaR95,P_CVaR97,P_CVaR98,P_MAD1,P_MAD2,P_MAD3,P_N
year,,,,,,,,
2015,-6.91%,-6.46%,-6.78%,-7.37%,-5.08%,-5.20%,-5.10%,-2.09%
2016,5.87%,5.84%,5.76%,6.14%,7.67%,7.10%,7.26%,8.32%
2017,24.17%,24.33%,24.39%,23.96%,25.92%,25.13%,24.54%,21.57%
2018,5.75%,6.04%,5.95%,5.65%,4.46%,4.46%,4.39%,7.69%
2019,29.26%,28.88%,29.04%,29.41%,26.82%,27.35%,28.80%,25.06%
2020,27.67%,27.63%,27.52%,27.78%,26.25%,27.02%,26.95%,32.55%
2021,0.19%,0.66%,0.42%,0.15%,-2.42%,-2.26%,-1.80%,1.63%
